# ANC GCP: Minimal LangGraph Notebook

This notebook is the streamlined baseline for the GCP path. It keeps execution minimal while documenting architecture, guardrails, and next-phase plans.

## Vision
- Build a Wikipedia-grounded, citation-first pipeline with a simple LangGraph workflow.
- Keep this notebook runnable in PyCharm and hosted notebook environments.
- Use this only for proof-of-concept; feature expansion moves to the Flask localhost app next.

## Architecture Summary
- Entry point: `notebooks/anc_gcp.ipynb`
- Data source intent: Wikipedia text/metadata plus optional picture files sourced from Wikipedia/Wikimedia
- Provider intent: Vertex AI embeddings + LLM (production path), represented here as a provider stub
- Output intent: grounded answers with Wikipedia citations

## Guardrails
- Keep citations Wikipedia-only for factual claims.
- If evidence is weak, say so explicitly.
- Allow a small number of uploaded pictures only when files are sourced from Wikipedia/Wikimedia pages.
- Keep this notebook minimal and stable; future product features go into Flask.


## PlantUML Workflow Diagram

```plantuml
@startuml
title ANC GCP Minimal LangGraph
start
:Input question;
:prepare_context node;
:draft_answer node;
:finalize_answer node;
:Return answer + citation placeholder;
stop
@enduml
```


In [ ]:
# Uncomment in fresh environments:
# %pip install -q -r ../requirements-gcp-dev.txt


In [ ]:
from typing import TypedDict

from langgraph.graph import END, START, StateGraph


class GraphState(TypedDict):
    question: str
    provider: str
    context: str
    draft: str
    answer: str


def prepare_context(state: GraphState) -> GraphState:
    context = (
        f"{state['provider']} context stub for: {state['question']}. "
        "In full mode this would attach retrieved Wikipedia snippets with citations."
    )
    return {**state, 'context': context}


def draft_answer(state: GraphState) -> GraphState:
    draft = f"Draft answer based on context: {state['context']}"
    return {**state, 'draft': draft}


def finalize_answer(state: GraphState) -> GraphState:
    answer = (
        f"{state['draft']} Citation placeholder: "
        "https://en.wikipedia.org/wiki/Main_Page"
    )
    return {**state, 'answer': answer}


graph = StateGraph(GraphState)
graph.add_node('prepare_context', prepare_context)
graph.add_node('draft_answer', draft_answer)
graph.add_node('finalize_answer', finalize_answer)

graph.add_edge(START, 'prepare_context')
graph.add_edge('prepare_context', 'draft_answer')
graph.add_edge('draft_answer', 'finalize_answer')
graph.add_edge('finalize_answer', END)

app = graph.compile()
print('Graph compiled successfully.')


In [ ]:
result = app.invoke(
    {
        'question': 'What states does Interstate 81 run through?',
        'provider': 'GCP/Vertex AI',
        'context': '',
        'draft': '',
        'answer': '',
    }
)

for key in ('provider', 'question', 'context', 'draft', 'answer'):
    print(f"{key}: {result[key]}")


## Discussion and Next Phases

This notebook demonstrates only core LangGraph wiring. After this baseline is stable in PyCharm and cloud notebooks, new functionality should move to the localhost Flask proof-of-concept application.

Planned progression:
1. Keep notebook scope fixed as a minimal reference harness.
2. Complete all notebook baselines with Wikipedia/Wikimedia-only data and picture sources.
3. Build and iterate core product behavior in Flask on localhost.
4. Expand app/flask data sources to additional public sources (for example USGS).
5. Begin iOS planning once Flask functionality is mature and stable.
